# MAMOB V2 — Trend Following Multi-Assets (CAC-like)

**Objectif** : construire un backtest *trend following* long-only multi-actifs, comparer **1-couple** vs **3-couples** de moyennes mobiles, et rendre l’évaluation **robuste** via walk-forward (**IS/OOS**) et rendre compte des diagnostics.

Ce notebook documente :
- la **démarche d’amélioration** depuis la V1,
- les **choix de modélisation**,
- les **difficultés** rencontrées et comment elles ont été résolues,
- les **résultats**, limites et pistes d’amélioration.

> Remarque : l’application Flask fournit l’interface + pages diagnostics.  
> Ici, le notebook sert à expliquer et démontrer la logique de bout en bout de manière “académique”.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

plt.rcParams["figure.figsize"] = (11, 4)
pd.set_option("display.max_columns", 50)

## 1) Point de départ (V1) et constats

### V1 (point de départ)
- Backtest “trend following” multi-actifs
- Signaux basés sur **moyennes mobiles (MA)**
- Sorties basiques : equity curve, performance

### Problèmes identifiés
1. **Choix des MA arbitraire** : risque de *data snooping* (sur-optimisation sur l’échantillon).
2. **Métriques incomplètes** : comparaison incomplète “rendement vs risque”.
3. **Cash restant vs fully-invested** : si la somme des expositions < 1, le portefeuille garde du cash résiduel, c'est incohérent si on veut une approche “no cash”.

## 2) Données (exemple reproductible)

Pour que le notebook reste lisible et rapide, on travaille sur un sous-univers (8 titres).  
L’app Flask permet de sélectionner un univers plus large.

In [ ]:
tickers = ["AI.PA","MC.PA","OR.PA","BNP.PA","SAN.PA","SU.PA","DG.PA","CAP.PA"]

prices = yf.download(
    tickers,
    period="15y",
    auto_adjust=False,
    progress=False,
    group_by="column"
)["Close"].dropna(how="all")

prices.tail()

## 3) Clarifier la logique de la stratégie

### Signal
- **Haussier** si `MA_short > MA_long` ⇒ signal = 1
- Sinon signal = 0

### Difficulté critique : look-ahead bias
Si on trade le même jour que le signal calculé avec la clôture du jour, on introduit de l’anticipation (“triche” involontaire).

✅ Solution : `shift(1)`  
On calcule le signal à *t* mais on l’applique au trading à *t+1*.

In [ ]:
def ma_signal(prices: pd.DataFrame, short_w: int, long_w: int) -> pd.DataFrame:
    ma_s = prices.rolling(short_w).mean()
    ma_l = prices.rolling(long_w).mean()
    sig = (ma_s > ma_l).astype(int)
    return sig.shift(1).fillna(0)  # évite look-ahead

signal_20_100 = ma_signal(prices, 20, 100)
signal_20_100.tail()

## 4) Cash vs Fully-invested (no cash) : normalisation des poids

### Problème
Avec un signal binaire 0/1, la somme des expositions varie dans le temps :
- si 3 actions sont haussières et 5 baissières, l’exposition brute ressemble à `[1,1,1,0,0,0,0,0]` (somme = 3)
- ce n’est **pas** une allocation : il manque la contrainte **somme des poids = 1**

### Choix de design (V2)
**Objectif : portefeuille toujours 100% investi (cash = 0)**

1) Normaliser chaque jour :
\[
w_i(t) = \frac{exposure_i(t)}{\sum_j exposure_j(t)}
\]

2) Cas critique : \(\sum exposure = 0\) (tout bearish)
- on ne peut pas normaliser
- et passer brutalement en cash augmente le churn (et ne respecte pas le “no cash”)

✅ Fallback retenu : **garder le portefeuille de la veille** (moins de churn)  
- si c’est le tout début (pas de veille) : **equal-weight**.

In [ ]:
def normalize_weights_no_cash_with_fallback(exposure: pd.DataFrame) -> pd.DataFrame:
    exposure = exposure.clip(lower=0)
    row_sum = exposure.sum(axis=1)

    # normalisation (jours où row_sum > 0)
    w = exposure.div(row_sum.replace(0, np.nan), axis=0)

    # fallback : si row_sum=0 => on conserve les poids précédents (moins de churn)
    w = w.ffill()

    # tout début (pas de poids précédent) => égal-pondéré
    w = w.fillna(1.0 / exposure.shape[1])
    return w

weights_1 = normalize_weights_no_cash_with_fallback(signal_20_100)
weights_1.tail()

## 5) Backtest (1-couple) — preuve de concept

On calcule les rendements du portefeuille :
\[
r_p(t) = \sum_i w_i(t)\, r_i(t)
\]
et la courbe d’equity :
\[
Equity(t) = \prod_{u \le t} (1 + r_p(u))
\]

In [ ]:
returns = prices.pct_change().fillna(0)

port_ret_1 = (weights_1 * returns).sum(axis=1)
equity_1 = (1 + port_ret_1).cumprod()

equity_1.plot(title="Equity curve — Trend 1-couple (20/100)")
plt.show()

## 6) 3 couples de MA vs 1 couple : logique + test

### Intuition “3 MA”
- 3 couples = 3 horizons (fast / medium / slow)
- conviction = moyenne des 3 signaux ⇒ signal plus lissé (0, 1/3, 2/3, 1)
- objectif : réduire la sensibilité à un seul paramètre

Mais on ne s’arrête pas à la théorie : on compare 1-couple vs 3-couples sur les mêmes métriques.

In [ ]:
triple = [(12,60), (24,120), (48,200)]
signals = [ma_signal(prices, s, l) for (s, l) in triple]
exposure_3 = sum(signals) / len(signals)

weights_3 = normalize_weights_no_cash_with_fallback(exposure_3)
port_ret_3 = (weights_3 * returns).sum(axis=1)
equity_3 = (1 + port_ret_3).cumprod()

pd.DataFrame({"Trend 1-couple": equity_1, "Trend 3-couples": equity_3}).dropna().plot(
    title="Equity curves — 1-couple vs 3-couples"
)
plt.show()

## 7) Ajouter des métriques “pro” (comparaison risk-adjusted)

Pour comparer proprement (au-delà de “ça monte”), on utilise :
- Perf totale
- CAGR (annualisé)
- Vol annualisée
- Sharpe (rf=0)
- Max drawdown (MDD)
- Turnover (proxy de coûts de transaction / churn)

> Dans l’app Flask, on ajoute aussi beta/alpha vs B&H et des diagnostics IS/OOS.

In [ ]:
def annualize_return(equity: pd.Series) -> float:
    if len(equity) < 2:
        return float("nan")
    total = equity.iloc[-1] / equity.iloc[0] - 1
    return (1 + total) ** (252 / len(equity)) - 1

def annualize_vol(ret: pd.Series) -> float:
    return float(ret.std() * np.sqrt(252))

def sharpe_ratio(ann_ret: float, ann_vol: float) -> float:
    if ann_vol == 0 or np.isnan(ann_vol):
        return float("nan")
    return float(ann_ret / ann_vol)

def max_drawdown(equity: pd.Series) -> float:
    peak = equity.cummax()
    dd = equity / peak - 1
    return float(dd.min())

def turnover_from_weights(weights: pd.DataFrame) -> pd.Series:
    return (weights - weights.shift(1).fillna(0)).abs().sum(axis=1)

def kpis(name: str, equity: pd.Series, ret: pd.Series, weights: pd.DataFrame) -> pd.Series:
    total = equity.iloc[-1] / equity.iloc[0] - 1
    ann = annualize_return(equity)
    vol = annualize_vol(ret)
    sh = sharpe_ratio(ann, vol)
    mdd = max_drawdown(equity)
    to_mean = float(turnover_from_weights(weights).mean())
    return pd.Series(
        {"Total Return": total, "CAGR": ann, "Vol": vol, "Sharpe": sh, "MaxDD": mdd, "Turnover_mean": to_mean},
        name=name
    )

kpi_table = pd.concat([
    kpis("Trend 1-couple (20/100)", equity_1, port_ret_1, weights_1),
    kpis("Trend 3-couples (12/60|24/120|48/200)", equity_3, port_ret_3, weights_3),
], axis=1).T

kpi_table

## 8) Walk-forward / IS-OOS (V2) : rendre la sélection “défendable”

### Problème
Choisir des MA “au hasard” (ou après avoir regardé les résultats) revient à faire du *data snooping*.

### Solution (V2)
Une procédure simple, claire, et défendable :
- période choisie (1Y/2Y/5Y/10Y/15Y)
- split :
  - **70% calibration (IS)**
  - **30% test (OOS)**
- test d’une grille de candidats (ex. ~15 couples, ~15 triples)
- sélection sur IS (dans ton code : **Sharpe IS**)
- évaluation sur OOS : la vraie performance hors-échantillon

➡️ “On optimise sur le passé proche, on juge sur le futur”.

### Démonstration (option) : réutiliser le moteur de l’app Flask

Si `app.py` est dans le même dossier que ce notebook, on peut importer ses fonctions et lancer un walk-forward identique à l’application.

> Si l’import échoue, passe directement à la section suivante : ce notebook reste valable.

In [ ]:
try:
    import app as core  # réutilise ton moteur (sans lancer Flask)
    has_core = True
except Exception as e:
    has_core = False
    print("Import app.py impossible ici (normal si le notebook est ailleurs). Détail:", e)

has_core

In [ ]:
if has_core:
    prices_full = core.get_prices_multi(core.TICKERS[:8], "15y")
    best_pair, diag1, split_date, tested, total = core.walk_forward_select_best_pair(
        prices_full, core.MA_CANDIDATES_20_100
    )

    print("Best pair:", best_pair, "| split_date:", split_date.date(), "| tested:", tested, "/", total)
    diag1.head()

## 9) Audit-proof : stabilité OOS en 3 blocs

Même si le Sharpe OOS global est bon, il peut dépendre d’une seule sous-période.

✅ Amélioration V2 :
- découper l’OOS en **3 blocs consécutifs**
- calculer le Sharpe par bloc
- compter les blocs “valides” (≥ un nombre minimal de jours)

Cela répond à la critique :  
> “Ton résultat dépend peut-être d’une seule sous-période.”

In [ ]:
if has_core:
    s, l = best_pair
    res, w = core.backtest_trend_one_pair_no_cash(prices_full, s, l)
    oos = res.loc[res.index >= split_date].copy()

    stab = core.stability_metrics(oos)
    stab
else:
    print("Section audit-proof: nécessite l'import de app.py (optionnel).")

## 10) Difficultés rencontrées (et solutions)

1) **Look-ahead bias**  
- Problème : trading le même jour que le signal basé sur la clôture  
- Solution : `shift(1)`

2) **Cash vs fully-invested**  
- Problème : somme des expositions < 1 ⇒ cash implicite  
- Solution : normalisation + fallback si somme=0 (keep last weights)

3) **Data snooping / sur-optimisation des paramètres**  
- Problème : choisir MA après observation des résultats  
- Solution : walk-forward IS/OOS + grille de candidats

4) **Périodes courtes / contraintes d’historique**  
- Problème : MA longues impossibles sur 1Y/2Y (OOS trop court, MA pas “stable”)  
- Solution : filtres de faisabilité + messages d’erreur clairs (dans l’app)

5) **Churn / turnover**  
- Problème : rotation excessive (coûts et instabilité)  
- Solution : mesurer le turnover + fallback “keep last weights” réduit le churn

## 11) Limites et améliorations possibles

**Limites**
- Données Yahoo Finance : qualité variable, trous de données possibles.
- Pas de coûts explicites (turnover = proxy).
- Long-only, pas de gestion du cash ou de short.
- Sélection basée sur Sharpe IS : peut encore sur-optimiser.

**Améliorations**
- Ajouter des coûts proportionnels au turnover (slippage / commissions).
- Critère de sélection plus robuste : mix IS/OOS + stabilité par blocs.
- Tests sur univers plus large et périodes supplémentaires.
- Analyse de sensibilité et tests de robustesse (sous-échantillons / bootstraps).